# Solution 2: CKD Level Classification
## Objective: Classify CKD stages using predicted creatinine and eGFR calculations

### Approach:
1. Load predicted creatinine values from Solution 1
2. Calculate eGFR using CKD-EPI equation
3. Apply KDIGO classification for CKD stages (multiclass)
4. Train classification models:
   - Multi-Layer Perceptron (MLP)
   - Support Vector Machine (SVM)

### KDIGO CKD Stages:
- **G1**: eGFR ≥ 90 (Normal or high)
- **G2**: eGFR 60-89 (Mildly decreased)
- **G3a**: eGFR 45-59 (Mildly to moderately decreased)
- **G3b**: eGFR 30-44 (Moderately to severely decreased)
- **G4**: eGFR 15-29 (Severely decreased)
- **G5**: eGFR < 15 (Kidney failure)

In [ ]:
# Import necessary libraries
import pandas as pd
.+.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## 1. Load Data from Solution 1

In [ ]:
# Load predictions and data from Solution 1
with open('solution1_output.pkl', 'rb') as f:
    solution1_data = pickle.load(f)

X_test = solution1_data['X_test']
y_actual_creatinine = solution1_data['y_test_actual']
y_predicted_creatinine = solution1_data['y_test_predicted']
best_model_name = solution1_data['best_model_name']
df_original = solution1_data['df_original']
test_indices = solution1_data['test_indices']

print(f"Best Model from Solution 1: {best_model_name}")
print(f"\nTest Set Size: {len(X_test)}")
print(f"Predicted Creatinine Range: [{y_predicted_creatinine.min():.2f}, {y_predicted_creatinine.max():.2f}] mg/dL")
print(f"Actual Creatinine Range: [{y_actual_creatinine.min():.2f}, {y_actual_creatinine.max():.2f}] mg/dL")

## 2. Calculate eGFR using CKD-EPI Equation

The CKD-EPI equation requires:
- Serum creatinine (mg/dL)
- Age (years)
- Sex (male/female)
- Race (optional adjustment for African American)

**CKD-EPI Equation:**
- For females with Scr ≤ 0.7: eGFR = 144 × (Scr/0.7)^(-0.329) × 0.993^Age
- For females with Scr > 0.7: eGFR = 144 × (Scr/0.7)^(-1.209) × 0.993^Age
- For males with Scr ≤ 0.9: eGFR = 141 × (Scr/0.9)^(-0.411) × 0.993^Age
- For males with Scr > 0.9: eGFR = 141 × (Scr/0.9)^(-1.209) × 0.993^Age

In [ ]:
# Extract required data from original dataset
# We need age and sex for eGFR calculation
test_data = df_original.iloc[test_indices].copy()

# Decode byte strings if necessary
for col in test_data.columns:
    if test_data[col].dtype == object:
        try:
            test_data[col] = test_data[col].str.decode('utf-8')
        except:
            pass

# Handle missing values in sex column
test_data['sex'] = test_data['sex'].replace(['?', '\t?', ' ?', '? '], np.nan)
test_data['sex'] = test_data['sex'].fillna(test_data['sex'].mode()[0])

# Convert age to numeric
test_data['age'] = pd.to_numeric(test_data['age'], errors='coerce')
test_data['age'] = test_data['age'].fillna(test_data['age'].median())

print("Sex distribution:")
print(test_data['sex'].value_counts())
print(f"\nAge range: [{test_data['age'].min():.0f}, {test_data['age'].max():.0f}] years")

In [ ]:
def calculate_egfr_ckd_epi(creatinine, age, sex, race='non_african_american'):
    """
    Calculate eGFR using CKD-EPI equation
    
    Parameters:
    - creatinine: serum creatinine (mg/dL)
    - age: age in years
    - sex: 'male' or 'female'
    - race: 'african_american' or 'non_african_american'
    
    Returns:
    - eGFR in mL/min/1.73m²
    """
    # Normalize sex values
    sex = str(sex).strip().lower()
    
    if sex in ['female', 'f', 'fem']:
        kappa = 0.7
        alpha = -0.329
        sex_factor = 144
        if creatinine > 0.7:
            alpha = -1.209
    else:  # male
        kappa = 0.9
        alpha = -0.411
        sex_factor = 141
        if creatinine > 0.9:
            alpha = -1.209
    
    # Calculate eGFR
    egfr = sex_factor * ((creatinine / kappa) ** alpha) * (0.993 ** age)
    
    # Race adjustment (1.159 multiplier for African American)
    if race == 'african_american':
        egfr *= 1.159
    
    return egfr

# Calculate eGFR for predicted creatinine values
egfr_predicted = []
for idx, (scr, age, sex) in enumerate(zip(y_predicted_creatinine, 
                                            test_data['age'].values,
                                            test_data['sex'].values)):
    egfr = calculate_egfr_ckd_epi(scr, age, sex)
    egfr_predicted.append(egfr)

egfr_predicted = np.array(egfr_predicted)

# Calculate eGFR for actual creatinine values (ground truth)
egfr_actual = []
for idx, (scr, age, sex) in enumerate(zip(y_actual_creatinine.values,
                                           test_data['age'].values,
                                           test_data['sex'].values)):
    egfr = calculate_egfr_ckd_epi(scr, age, sex)
    egfr_actual.append(egfr)

egfr_actual = np.array(egfr_actual)

print(f"Predicted eGFR Range: [{egfr_predicted.min():.2f}, {egfr_predicted.max():.2f}] mL/min/1.73m²")
print(f"Actual eGFR Range: [{egfr_actual.min():.2f}, {egfr_actual.max():.2f}] mL/min/1.73m²")

## 3. Apply KDIGO Classification

In [ ]:
def classify_ckd_stage(egfr):
    """
    Classify CKD stage based on KDIGO guidelines
    
    Parameters:
    - egfr: estimated GFR (mL/min/1.73m²)
    
    Returns:
    - CKD stage (G1, G2, G3a, G3b, G4, G5)
    """
    if egfr >= 90:
        return 'G1'
    elif egfr >= 60:
        return 'G2'
    elif egfr >= 45:
        return 'G3a'
    elif egfr >= 30:
        return 'G3b'
    elif egfr >= 15:
        return 'G4'
    else:
        return 'G5'

# Classify CKD stages
ckd_stages_predicted = np.array([classify_ckd_stage(egfr) for egfr in egfr_predicted])
ckd_stages_actual = np.array([classify_ckd_stage(egfr) for egfr in egfr_actual])

print("Predicted CKD Stage Distribution:")
print(pd.Series(ckd_stages_predicted).value_counts().sort_index())
print("\nActual CKD Stage Distribution:")
print(pd.Series(ckd_stages_actual).value_counts().sort_index())

In [ ]:
# Visualize eGFR and CKD stage distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# eGFR distribution
axes[0, 0].hist(egfr_actual, bins=30, alpha=0.5, label='Actual', edgecolor='black')
axes[0, 0].hist(egfr_predicted, bins=30, alpha=0.5, label='Predicted', edgecolor='black')
axes[0, 0].set_xlabel('eGFR (mL/min/1.73m²)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('eGFR Distribution')
axes[0, 0].legend()

# eGFR scatter plot
axes[0, 1].scatter(egfr_actual, egfr_predicted, alpha=0.5)
axes[0, 1].plot([0, 150], [0, 150], 'r--', lw=2)
axes[0, 1].set_xlabel('Actual eGFR (mL/min/1.73m²)')
axes[0, 1].set_ylabel('Predicted eGFR (mL/min/1.73m²)')
axes[0, 1].set_title('Actual vs Predicted eGFR')

# CKD stage distribution
stage_order = ['G1', 'G2', 'G3a', 'G3b', 'G4', 'G5']
actual_counts = pd.Series(ckd_stages_actual).value_counts().reindex(stage_order, fill_value=0)
predicted_counts = pd.Series(ckd_stages_predicted).value_counts().reindex(stage_order, fill_value=0)

x = np.arange(len(stage_order))
width = 0.35

axes[1, 0].bar(x - width/2, actual_counts, width, label='Actual', alpha=0.7)
axes[1, 0].bar(x + width/2, predicted_counts, width, label='Predicted', alpha=0.7)
axes[1, 0].set_xlabel('CKD Stage')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('CKD Stage Distribution')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(stage_order)
axes[1, 0].legend()

# eGFR vs Age
axes[1, 1].scatter(test_data['age'], egfr_actual, alpha=0.5, label='Actual eGFR')
axes[1, 1].scatter(test_data['age'], egfr_predicted, alpha=0.5, label='Predicted eGFR')
axes[1, 1].set_xlabel('Age (years)')
axes[1, 1].set_ylabel('eGFR (mL/min/1.73m²)')
axes[1, 1].set_title('eGFR vs Age')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 4. Prepare Data for Classification Models

We'll use the original features plus the predicted creatinine to train classification models

In [ ]:
# Prepare full dataset with eGFR and CKD stages
# We'll use the entire original dataset for classification
df_full = df_original.copy()

# Decode byte strings
for col in df_full.columns:
    if df_full[col].dtype == object:
        try:
            df_full[col] = df_full[col].str.decode('utf-8')
        except:
            pass

# Handle missing values
df_full = df_full.replace(['?', '\t?', ' ?', '? '], np.nan)

# Convert numeric columns
numeric_cols = ['age', 'al', 'ph', 'sc']
for col in numeric_cols:
    df_full[col] = pd.to_numeric(df_full[col], errors='coerce')

# Impute missing values
from sklearn.impute import SimpleImputer
numeric_imputer = SimpleImputer(strategy='median')
df_full[numeric_cols] = numeric_imputer.fit_transform(df_full[numeric_cols])

categorical_cols = ['rbc', 'pc', 'ba', 'sex']
categorical_imputer = SimpleImputer(strategy='most_frequent')
df_full[categorical_cols] = categorical_imputer.fit_transform(df_full[categorical_cols])

print("Full Dataset Shape:", df_full.shape)
print("Missing Values:", df_full[numeric_cols + categorical_cols].isnull().sum().sum())

In [ ]:
# Calculate eGFR for all samples
egfr_full = []
for idx, row in df_full.iterrows():
    egfr = calculate_egfr_ckd_epi(row['sc'], row['age'], row['sex'])
    egfr_full.append(egfr)

df_full['egfr'] = egfr_full
df_full['ckd_stage'] = [classify_ckd_stage(egfr) for egfr in egfr_full]

print("\nCKD Stage Distribution (Full Dataset):")
print(df_full['ckd_stage'].value_counts().sort_index())

In [ ]:
# Encode categorical features
label_encoders_clf = {}
for col in ['rbc', 'pc', 'ba', 'sex']:
    le = LabelEncoder()
    df_full[col + '_encoded'] = le.fit_transform(df_full[col])
    label_encoders_clf[col] = le

# Encode target (CKD stage)
le_target = LabelEncoder()
df_full['ckd_stage_encoded'] = le_target.fit_transform(df_full['ckd_stage'])

print("CKD Stage Encoding:")
for i, label in enumerate(le_target.classes_):
    print(f"  {label} -> {i}")

In [ ]:
# Prepare features and target for classification
# Features: Age, Albumin, RBC, Pus Cell, Bacteria, pH, and Creatinine
feature_columns_clf = ['age', 'al', 'ph', 'sc', 'rbc_encoded', 'pc_encoded', 'ba_encoded', 'sex_encoded']
X_clf = df_full[feature_columns_clf].copy()
y_clf = df_full['ckd_stage_encoded'].copy()

print("Classification Feature Matrix Shape:", X_clf.shape)
print("Classification Target Vector Shape:", y_clf.shape)
print("\nNumber of Classes:", len(le_target.classes_))
print("Classes:", le_target.classes_)

In [ ]:
# Split data
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42, stratify=y_clf
)

print("Training Set Size:", X_train_clf.shape)
print("Test Set Size:", X_test_clf.shape)
print("\nTrain Class Distribution:")
print(pd.Series(y_train_clf).value_counts().sort_index())
print("\nTest Class Distribution:")
print(pd.Series(y_test_clf).value_counts().sort_index())

In [ ]:
# Feature scaling
scaler_clf = StandardScaler()
X_train_clf_scaled = scaler_clf.fit_transform(X_train_clf)
X_test_clf_scaled = scaler_clf.transform(X_test_clf)

print("Scaled Training Set Shape:", X_train_clf_scaled.shape)
print("Scaled Test Set Shape:", X_test_clf_scaled.shape)

## 5. Multi-Layer Perceptron (MLP) Classification

In [ ]:
# Train MLP Classifier
mlp_model = MLPClassifier(
    hidden_layer_sizes=(100, 50, 25),  # Three hidden layers
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    verbose=False
)

mlp_model.fit(X_train_clf_scaled, y_train_clf)

# Predictions
y_pred_mlp_train = mlp_model.predict(X_train_clf_scaled)
y_pred_mlp_test = mlp_model.predict(X_test_clf_scaled)

# Evaluation
print("MLP Classifier Performance:")
print("\nTraining Set:")
print(f"  Accuracy: {accuracy_score(y_train_clf, y_pred_mlp_train):.4f}")
print(f"  Precision (macro): {precision_score(y_train_clf, y_pred_mlp_train, average='macro', zero_division=0):.4f}")
print(f"  Recall (macro): {recall_score(y_train_clf, y_pred_mlp_train, average='macro'):.4f}")
print(f"  F1-Score (macro): {f1_score(y_train_clf, y_pred_mlp_train, average='macro'):.4f}")

print("\nTest Set:")
print(f"  Accuracy: {accuracy_score(y_test_clf, y_pred_mlp_test):.4f}")
print(f"  Precision (macro): {precision_score(y_test_clf, y_pred_mlp_test, average='macro', zero_division=0):.4f}")
print(f"  Recall (macro): {recall_score(y_test_clf, y_pred_mlp_test, average='macro'):.4f}")
print(f"  F1-Score (macro): {f1_score(y_test_clf, y_pred_mlp_test, average='macro'):.4f}")

In [ ]:
# Detailed classification report for MLP
print("\nMLP Classification Report (Test Set):")
print(classification_report(y_test_clf, y_pred_mlp_test, 
                          target_names=le_target.classes_,
                          zero_division=0))

In [ ]:
# Confusion Matrix for MLP
cm_mlp = confusion_matrix(y_test_clf, y_pred_mlp_test)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_mlp, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le_target.classes_,
            yticklabels=le_target.classes_)
plt.xlabel('Predicted CKD Stage')
plt.ylabel('Actual CKD Stage')
plt.title('MLP Confusion Matrix')
plt.show()

## 6. Support Vector Machine (SVM) Classification

In [ ]:
# Train SVM Classifier
svm_model = SVC(
    kernel='rbf',
    C=10,
    gamma='scale',
    random_state=42,
    probability=True  # Enable probability estimates
)

svm_model.fit(X_train_clf_scaled, y_train_clf)

# Predictions
y_pred_svm_train = svm_model.predict(X_train_clf_scaled)
y_pred_svm_test = svm_model.predict(X_test_clf_scaled)

# Evaluation
print("SVM Classifier Performance:")
print("\nTraining Set:")
print(f"  Accuracy: {accuracy_score(y_train_clf, y_pred_svm_train):.4f}")
print(f"  Precision (macro): {precision_score(y_train_clf, y_pred_svm_train, average='macro', zero_division=0):.4f}")
print(f"  Recall (macro): {recall_score(y_train_clf, y_pred_svm_train, average='macro'):.4f}")
print(f"  F1-Score (macro): {f1_score(y_train_clf, y_pred_svm_train, average='macro'):.4f}")

print("\nTest Set:")
print(f"  Accuracy: {accuracy_score(y_test_clf, y_pred_svm_test):.4f}")
print(f"  Precision (macro): {precision_score(y_test_clf, y_pred_svm_test, average='macro', zero_division=0):.4f}")
print(f"  Recall (macro): {recall_score(y_test_clf, y_pred_svm_test, average='macro'):.4f}")
print(f"  F1-Score (macro): {f1_score(y_test_clf, y_pred_svm_test, average='macro'):.4f}")

In [ ]:
# Detailed classification report for SVM
print("\nSVM Classification Report (Test Set):")
print(classification_report(y_test_clf, y_pred_svm_test, 
                          target_names=le_target.classes_,
                          zero_division=0))

In [ ]:
# Confusion Matrix for SVM
cm_svm = confusion_matrix(y_test_clf, y_pred_svm_test)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Greens',
            xticklabels=le_target.classes_,
            yticklabels=le_target.classes_)
plt.xlabel('Predicted CKD Stage')
plt.ylabel('Actual CKD Stage')
plt.title('SVM Confusion Matrix')
plt.show()

## 7. Model Comparison

In [ ]:
# Compare models
results_clf = pd.DataFrame({
    'Model': ['MLP', 'SVM'],
    'Train_Accuracy': [
        accuracy_score(y_train_clf, y_pred_mlp_train),
        accuracy_score(y_train_clf, y_pred_svm_train)
    ],
    'Test_Accuracy': [
        accuracy_score(y_test_clf, y_pred_mlp_test),
        accuracy_score(y_test_clf, y_pred_svm_test)
    ],
    'Train_F1': [
        f1_score(y_train_clf, y_pred_mlp_train, average='macro'),
        f1_score(y_train_clf, y_pred_svm_train, average='macro')
    ],
    'Test_F1': [
        f1_score(y_test_clf, y_pred_mlp_test, average='macro'),
        f1_score(y_test_clf, y_pred_svm_test, average='macro')
    ]
})

print("Classification Model Comparison:")
print(results_clf)

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

results_clf.plot(x='Model', y=['Train_Accuracy', 'Test_Accuracy'], kind='bar', ax=axes[0], alpha=0.7)
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Model Comparison - Accuracy')
axes[0].legend(['Train Accuracy', 'Test Accuracy'])
axes[0].set_xticklabels(results_clf['Model'], rotation=0)
axes[0].set_ylim([0, 1])

results_clf.plot(x='Model', y=['Train_F1', 'Test_F1'], kind='bar', ax=axes[1], alpha=0.7)
axes[1].set_ylabel('F1-Score (Macro)')
axes[1].set_title('Model Comparison - F1-Score')
axes[1].legend(['Train F1', 'Test F1'])
axes[1].set_xticklabels(results_clf['Model'], rotation=0)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

## 8. Cross-Validation Analysis

In [ ]:
# Perform cross-validation for both models
from sklearn.model_selection import cross_validate

cv_scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# MLP Cross-Validation
print("MLP 5-Fold Cross-Validation:")
mlp_cv = cross_validate(mlp_model, X_train_clf_scaled, y_train_clf, 
                        cv=5, scoring=cv_scoring, n_jobs=-1)
print(f"  Accuracy: {mlp_cv['test_accuracy'].mean():.4f} (+/- {mlp_cv['test_accuracy'].std():.4f})")
print(f"  Precision: {mlp_cv['test_precision_macro'].mean():.4f} (+/- {mlp_cv['test_precision_macro'].std():.4f})")
print(f"  Recall: {mlp_cv['test_recall_macro'].mean():.4f} (+/- {mlp_cv['test_recall_macro'].std():.4f})")
print(f"  F1-Score: {mlp_cv['test_f1_macro'].mean():.4f} (+/- {mlp_cv['test_f1_macro'].std():.4f})")

# SVM Cross-Validation
print("\nSVM 5-Fold Cross-Validation:")
svm_cv = cross_validate(svm_model, X_train_clf_scaled, y_train_clf,
                        cv=5, scoring=cv_scoring, n_jobs=-1)
print(f"  Accuracy: {svm_cv['test_accuracy'].mean():.4f} (+/- {svm_cv['test_accuracy'].std():.4f})")
print(f"  Precision: {svm_cv['test_precision_macro'].mean():.4f} (+/- {svm_cv['test_precision_macro'].std():.4f})")
print(f"  Recall: {svm_cv['test_recall_macro'].mean():.4f} (+/- {svm_cv['test_recall_macro'].std():.4f})")
print(f"  F1-Score: {svm_cv['test_f1_macro'].mean():.4f} (+/- {svm_cv['test_f1_macro'].std():.4f})")

## 9. Feature Importance Analysis

In [ ]:
# Analyze feature importance using permutation importance
from sklearn.inspection import permutation_importance

# MLP Feature Importance
mlp_perm_importance = permutation_importance(
    mlp_model, X_test_clf_scaled, y_test_clf, 
    n_repeats=10, random_state=42, n_jobs=-1
)

mlp_importance_df = pd.DataFrame({
    'Feature': feature_columns_clf,
    'Importance': mlp_perm_importance.importances_mean
}).sort_values('Importance', ascending=False)

print("MLP Feature Importance (Permutation):")
print(mlp_importance_df)

plt.figure(figsize=(10, 6))
plt.barh(mlp_importance_df['Feature'], mlp_importance_df['Importance'])
plt.xlabel('Importance')
plt.title('MLP Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# SVM Feature Importance
svm_perm_importance = permutation_importance(
    svm_model, X_test_clf_scaled, y_test_clf,
    n_repeats=10, random_state=42, n_jobs=-1
)

svm_importance_df = pd.DataFrame({
    'Feature': feature_columns_clf,
    'Importance': svm_perm_importance.importances_mean
}).sort_values('Importance', ascending=False)

print("SVM Feature Importance (Permutation):")
print(svm_importance_df)

plt.figure(figsize=(10, 6))
plt.barh(svm_importance_df['Feature'], svm_importance_df['Importance'])
plt.xlabel('Importance')
plt.title('SVM Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 10. Summary and Conclusion

In [ ]:
print("=" * 70)
print("SOLUTION 2: CKD LEVEL CLASSIFICATION - SUMMARY")
print("=" * 70)
print(f"\nDataset Size: {len(df_full)} samples")
print(f"Features Used: {len(feature_columns_clf)} features")
print(f"  - Numeric: Age, Albumin, Urine pH, Serum Creatinine")
print(f"  - Categorical (encoded): RBC, Pus Cell, Bacteria, Sex")
print(f"\neGFR Calculation: CKD-EPI equation")
print(f"Classification: KDIGO CKD stages (G1-G5)")
print(f"Number of Classes: {len(le_target.classes_)}")
print(f"Classes: {', '.join(le_target.classes_)}")
print(f"\nTrain/Test Split: 80/20")
print(f"Training Samples: {len(X_train_clf)}")
print(f"Test Samples: {len(X_test_clf)}")
print("\n" + "=" * 70)
print("MODEL PERFORMANCE SUMMARY")
print("=" * 70)
print(results_clf.to_string(index=False))
print("\n" + "=" * 70)
best_clf_idx = results_clf['Test_Accuracy'].idxmax()
best_clf_name = results_clf.loc[best_clf_idx, 'Model']
print(f"BEST CLASSIFICATION MODEL: {best_clf_name}")
print("=" * 70)
print(f"Test Accuracy: {results_clf.loc[best_clf_idx, 'Test_Accuracy']:.4f}")
print(f"Test F1-Score: {results_clf.loc[best_clf_idx, 'Test_F1']:.4f}")
print("\n" + "=" * 70)
print("PIPELINE COMPLETE")
print("=" * 70)
print(f"Solution 1 (Creatinine Prediction): {best_model_name} model")
print(f"Solution 2 (CKD Classification): {best_clf_name} model")
print("=" * 70)